
<h1 style="color: rgb(241, 90, 36)"><img src="./images/SQLIcon.png?modified=23223" width=80px height=80px style="vertical-align: middle;"> Types of Subqueries</h2>

There are various types of subqueries that can be used to get the results you want.

<h3 style="color: rgb(241, 90, 36)"> Single row and scalar subqueries</h3>

These types of subqueries either return a **single row** or *scalar* value. Some common uses for them are:

- Updating, deleting or inserting data based on the row returned by the subquery
- Returning a **scalar** value which can be used as a comparison in your `HAVING` and `WHERE` statements
- Setting a default value for columns in your `SELECT` statement in the case of a `NULL`
- Updating a single value in your table

Let's look at some examples. Imagine we wanted to get all films from the `film` table which have a greater than average `runtime`. We could do so with the following query:

>All example code will be run on the **pagila** database, so make sure you have it available to run the code in this notebook.

In [ ]:
SELECT title,
       length
FROM
    film
WHERE
    length > 
    (
        SELECT AVG(length)
        FROM film
    )
ORDER BY 
    length DESC;

Which returns all movies with a `length` greater than the **average** runtime, note here our subquery:

In [ ]:
(
    SELECT AVG(length)
    FROM film
);

Returns a scalar value `115.272`. Since subqueries are evaluated inside out, imagine the `WHERE` statement is evaluated, then it reads: 

`WHERE length > 115.272` 

So it's filtering the table for rows where `length > 115.272`. All the usual **comparison operators** would have worked here for a scalar subquery `<, >, =`, `>=`, `<=` or `<>`. Ordering the data we get the following result:

<img src='images/average_length_subquery.png?modified=343232'>

Now what if we wanted to display the **average length** alongside the `length` column to visually validate our data. Then we could actually create a new column based on a subquery inside the `SELECT` statement.

In [ ]:
SELECT title,
       length,
       (
            SELECT ROUND(AVG(length)) AS average_film_length
            FROM film
       )
FROM
    film
WHERE
    length > 
    (
        SELECT AVG(length)
        FROM film
    )
ORDER BY 
    length;

Returning the results:

<img src='images/scalar_sub_select.png?modified=32232'>

The `length` column has been sorted in **descending** order this time, to show that all movies in the resultant query are in fact over the **average** length. 

> So we can actually use subqueries in `SELECT` statements to generate new columns if they return a scalar value.

We can also use a subquery in the `HAVING` statement to filter the data:

In [ ]:
SELECT AVG(length) AS film_rating_average_length,
       rating
FROM
    film
GROUP BY
    rating
HAVING
    AVG(length) > 
    (
        SELECT AVG(length)
        FROM film
    );

Here we've filtered the results after the `GROUP BY` statement using the `HAVING` statement. Let's group the films by `rating` and then determine which `rating`s have a higher than average `replacement_cost`. 

In [ ]:
SELECT rating,
       ROUND(AVG(replacement_cost), 2) AS avg_rating_replacement_cost,
       (
            SELECT ROUND(AVG(replacement_cost),2) AS avg_film_replacement_cost
            FROM film
       )
FROM
    film
GROUP BY
    rating
HAVING
    AVG(replacement_cost) > 
    (
        SELECT AVG(replacement_cost)
        FROM film
    );

We've used another subquery to create a column with the average `replacement_cost` of all movies, so we can validate the data. In the `HAVING` statement we're checking that the average `replacement_cost` for that specific rating is greater than the average `replacement_cost` across all movies. This gives the following results:

<img src='images/avg_replacement_cost.png?modified=204524'>

<h3 style="color: rgb(241, 90, 36)"> Multiple row subquery</h3>

Subqueries that return multiple rows of data to an **outside query** are called *multiple-row subqueries*. These are great for targeting a **subsets** of the data. They can be used in the `SELECT`, `FROM`, `WHERE` and `HAVING` just like *single row subqueries*. They can't be used with **comparison operators**, unless preceded with one of the **keywords** `ANY`, `ALL`, `IN` or `NOT IN`.

Let's imagine we want to check which actors appeared in the film `ACE GOLDFINGER` in the `film` table, which has `film_id = 2`. We could do this by joining the two tables and then filtering the data in the `WHERE` statement. Or we could use a subquery to get the data:


In [ ]:
SELECT actor_id,
       first_name,
       last_name
FROM 
    actor
WHERE 
    actor_id IN 
    (
        SELECT actor_id
        FROM film_actor
        WHERE film_id = 2
    );

First we evaluate the inner query which selects all rows from the `film_actor` table where the `film_id = 2`. This should give us a **subset** of the `film_actor` table since there are multiple rows meeting the condition. You can check this by running the subquery as its own query, it should produce the following result:



<img src='images/multi_row_subquery_run.png?modified=234324'>

So after the subquery is evaluated, the `WHERE` statement would then become `WHERE actor_id IN (19, 85, 90, 160)` which allows us to produce the final result.

<img src='images/subset_filter_multi_row.png?modified=23424'>

Ideally, we would like to `SELECT` the `film_id` column to be part of the final result, so we can verify the accuracy of the results. Unfortunately, the `film_id` column exists in the `film_actor` table, which the subquery selects from and does not exist in `actor` table used by the **outer query**.

<h4 style="color: rgb(241, 90, 36)"> ANY, ALL, IN, NOT IN</h4>

`ANY`, `ALL`, `IN`, `NOT IN` can be used in conjunction with a **comparison operator**. In all cases the subquery must return **at least one column** to be used with these subquery expressions. They have the following use cases:

- <b style="color: rgb(241, 90, 36)">ANY/SOME</b>: `ANY` or `SOME` can be used interchangeably. If the evaluated is `True` for any row returned by the subquery, then the statement will ultimately evaluate to `True`. 
- <b style="color: rgb(241, 90, 36)">IN</b>: Evaluates to `True` if the expression matches any rows in the subquery
- <b style="color: rgb(241, 90, 36)">NOT IN</b>: Evaluates to `True` if the expression does not match any rows in the subquery
- <b style="color: rgb(241, 90, 36)">ALL</b>: The statement will evaluate to `True` if the expression matches all rows in the subquery


In [ ]:
{expression} {operator} [ ALL | ANY | IN | NOT IN] (subquery)

Let's take a look at some examples:

In [ ]:
SELECT title,
       release_year,
       replacement_cost
FROM 
    film
WHERE 
    replacement_cost IN 
    (
        SELECT DISTINCT(replacement_cost)
        FROM film
        WHERE replacement_cost < 12.99
        AND replacement_cost > 9.99
    );

Here the subquery gets all movies with `replacement_cost` < `12.99`. If we run it on its own we get the following result:

<img src='images/replacement_cost_sub.png?modified=2234'>

In the query we're using the `IN` keyword to return all movies which have one of these rental ratings.

In [ ]:
replacement_cost > ANY 
(
    SELECT DISTINCT(replacement_cost)
    FROM film
    WHERE replacement_cost < 12.99 
    AND replacement_cost > 9.99
);

This query is saying, if the film `replacement_cost` is greater than `ANY` of the results of the subquery, then return the movie. So that's all movies with `replacement_cost > 10.99 or 11.99` which equates to `replacement_cost > 11.99`. 

> Just a small note about `IN` and `ANY`, if we had stated `replacement_cost = ANY` then this would be logically equivalent to `replacement_cost IN` and can be used interchangeably. Both statements will be checking the current row for equality against values in the subquery. Though the use of `ANY` does give you use of other operators making it the more versatile option. 

In [ ]:
replacement_cost NOT IN 
(
    SELECT DISTINCT(replacement_cost)
    FROM film
    WHERE replacement_cost < 12.99
    AND replacement_cost > 9.99
);

This returns all movies where their `replacement_cost` is not one of `10,99, 11.99`. So would return all rows where `replacement_cost <= 11.99` and `replacement_cost => 10.99`.

In [ ]:
replacement_cost <= ALL
(
    SELECT DISTINCT(replacement_cost)
    FROM film
    WHERE replacement_cost < 12.99
    AND replacement_cost > 9.99
);

The `ALL` keyword is checking that the current row is less or equal to **all** of `10.99 and 11.99`. The query would then return all movies with `replacement_cost <= 10.99`. Why `10.99`?  Remember with the `ALL` keyword we need the `replacement_cost` condition to be `True` for **all** conditions for the row to be returned. 

It's clear this condition won't return values `> 11.99`. The rows which the query returns have a `replacement_cost` value of `10.99` and `9.99`. Let's see why this is true by checking the conditions:

- For `11.99`, `11.99 <= 11.99 = true` and `11.99 <= 10.99 = false` so `11.99` is not returned, since we get `False` in one case
- For `10.99`, `10.99 <= 11.99 = true` and `10.99 <= 10.99 = true` so the `10.99` value is returned, since `True1 for all
- For `9.99`, `9.99 <= 11.99 = true` and `9.99 <= 10.99 = true` which is also returned, since `True` for all

<h3 style="color: rgb(241, 90, 36)"> Multiple column subqueries</h3>

*Multiple column subqueries* are named as such because the **subquery** returns multiple columns. The query is usually used to filter or sort data in the **outer query** based on the results returned by the **subquery**. 

An example of a **multiple column subquery** which retrieves all films with the maximum `rental_cost` and `rental_duration`:

In [ ]:
SELECT title,
       rental_duration,
       rental_rate
FROM 
    film
WHERE
    (rental_duration, rental_rate) IN
    (
        SELECT MAX(rental_duration),
               MAX(rental_rate)
        FROM
            film
    );

The subquery returns the results:

<img src='images/maximum_rental_rate.png?modified=232'>

Since our **outer query** is checking that the `rental_rate` and `rental_duration` exist within the **subquery** results, it will return all films which match those values. Which is to say the outer query is returning all results where the `rental_duration` and `rental_rate` are both at their maximum. 

<h3 style="color: rgb(241, 90, 36)"> Correlated subqueries</h3>

*Correlated subqueries* are named as such, since the output of the **subquery** depends on the results form the **outside query**, hence the two queries are **correlated**. A **correlated subquery** refers to a column which is not in the table of its `FROM` clause. These subqueries are used within the `SELECT`, `WHERE`, `HAVING` and `FROM` clauses. Let's take a look at an example of using a **correlated query** in the `SELECT` statement:

In [ ]:
SELECT customer_id,
       first_name,
       last_name,
       (
            SELECT ROUND(AVG(amount), 2) AS average_customer_payment
            FROM payment 
            WHERE customer.customer_id = payment.customer_id
        )
FROM
    customer; 

In the `SELECT` statement we've used the subquery to return the average payment made by each customer and included this as a column in our results. We've used the subquery to get the average `amount` paid by each customer. Notice that in the `WHERE` statement of the subquery we've matched the columns on the `customer_id` from `payment` and `customer` columns. So, the **inner query** is referencing the **outer query**, this creates the **correlation** between the two tables. 

We could have performed the same query using a `JOIN`:

In [ ]:
SELECT customer.customer_id,
       first_name,
       last_name,
       ROUND(AVG(payment.amount), 2) AS average_customer_payment
FROM
    customer
INNER JOIN
    payment ON payment.customer_id = customer.customer_id
GROUP BY
    customer.customer_id, first_name, last_name
ORDER BY
    customer_id;

With **correlated queries** you can often get the same information using a `JOIN`. In most cases a `JOIN` is more efficient, as subqueries are executed on every row of the table whereas a `JOIN` is performed once. It is rare a subquery outperforms a `JOIN` so when given the choice, try and write the statement with a `JOIN`. 

<h3 style="color: rgb(241, 90, 36)"> Nested subqueries</h3>

*Nested subqueries* are queries which are **nested** inside other subqueries. Remember that queries are executed inside out, so if you have a subquery inside another subquery, the innermost query will be executed first:

In [ ]:
SELECT title,
       description
FROM
    film
WHERE 
	film_id IN 
	(
		SELECT film_id
		FROM film_actor
		WHERE actor_id IN 
		(
			SELECT actor_id
			FROM actor
			WHERE first_name = 'CHRISTIAN'
		)
	);

Let's take a look at what this **nested query** is performing, the **innermost query** is executed first:

In [ ]:
SELECT actor_id
FROM actor
WHERE first_name = 'CHRISTIAN'

Returning:

<img src='images/nested_christian_inner.png?modified=2232'>

The outside query then becomes:

In [ ]:
SELECT film_id
FROM film_actor
WHERE actor_id IN (10, 58, 61)

Which returns the results(only partial results are shown):

<img src='images/nested_outer_film_id.png?modified=32423'>

Then finally the **outer most query** results in:

In [ ]:
SELECT title,
       description
FROM
    film
WHERE 
	film_id IN (1, 9, 191, 236, ....)

This returns all movies where the actors `first_name` is `CHRISTIAN`.

<h3 style="color: rgb(241, 90, 36)"> Derived tables</h3>

A *derived table* is a table which is derived from an expression in the `FROM` clause of a SQL query. **Derived tables** can be useful for applying SQL logic to a **subset** or **aggregations** of a table. One important thing to remember when creating **derived tables** is, they need to have an aliases, such that they can be referenced in the outside query. An example of a query using a **derived table**:

In [ ]:
SELECT ROUND(AVG(payment_sums.sum_of_payments), 2) AS average_sum_of_payments
FROM
    (
        SELECT SUM(amount) AS sum_of_payments
        FROM 
            payment
        GROUP BY
            customer_id       
    ) AS payment_sums

Here the subquery derives a table from the `payments` table, grouping all the `customer_id`'s and summing all their payments. Notice the subquery table is also aliased as `payment_sums`, which the **outside query** can then reference to get the average of all customer payments. 

If we evaluate the **inside query** and add the `customer_id` column to it, it's clear it's summing up the total payments for all customers:

In [ ]:
    
SELECT SUM(amount) AS sum_of_payments,
       customer_id
FROM 
    payment
GROUP BY
    customer_id;

Which returns the results:

<img src='images/sum_of_all_payments.png?modified=2332'>

We then get the average and `ROUND` the results by referencing the table in the `SELECT` statement, `ROUND(AVG(payment_sums.sum_of_payments), 2)` finalising the query:

<img src='images/derived_table_results.png?modified=3234'>

## Key Takeaways

- You can often get the same information using a `JOIN` instead of a **correlated subquery**
- When deciding which subquery you need it can be helpful to think of what information you need to return. Is it a scalar value, multiple rows?
- Using a **multiple row subquery** gives you access to the powerful `ANY`, `ALL`, `NOT IN` and `IN` operators
- Using a subquery to create a **derived table** can be a quick way to begin performing a query on a subset of data
- Continually nesting subqueries can be a great way to target incredibly specific data


